# Notebook 1 : Creating your first Manus Task

> Prerequisites: Sign up for an account at https://manus.im/ and create your first API Key

Manus is designed to take high-level instructions and turn them into concrete work: writing and modifying code, calling tools and APIs, and coordinating multi-step tasks without constant human supervision. 

While the web app provides an intuitive interface for interactive use, the API unlocks the true power: embedding Manus capabilities directly into your own systems, scripts, and products.

## Overview

This notebook marks the beginning of your journey through the Manus ecosystem. We'll start with the fundamentals—calling the API directly, creating and inspecting tasks, polling for completion, and reading outputs. 

In this first notebook, we'll walk you through creating your first Manus task from scratch. 

We'll progress from simple queries to understanding the complete task lifecycle, building the essential knowledge you need for the notebooks that follow. By the end of this session, you'll have hands-on experience with the core Manus API patterns that form the building blocks for building your own bespoke automations.

## Environment and Authentication

Before interacting with Manus, your environment needs valid credentials and basic configuration. 

In this section, you will load your API key, validate connectivity, and make a simple test request to confirm everything is working.

In [46]:
import os
import requests
from dotenv import load_dotenv
from rich import print

# Load API key from .env file in repository root
load_dotenv()

# Configuration
API_KEY = os.getenv("MANUS_API_KEY")
BASE_URL = "https://api.manus.ai/v1"

# Validate API key is loaded
if not API_KEY:
    raise ValueError("⚠️  Please set your MANUS_API_KEY in the .env file at the repository root")



url = "https://api.manus.ai/v1/files"
response = requests.get(url, headers={"API_KEY": API_KEY})

print(response.json())

print("✓ API key loaded successfully!")
print(f"✓ Base URL: {BASE_URL}")

{'object': 'list'}

✓ API key loaded successfully!

✓ Base URL: https://api.manus.ai/v1

Once your environment is authenticated, you’re ready to create your first real Manus task and see how the API behaves when executing a simple instruction.

## Creating Your First Task

A task is the fundamental unit of work in Manus. 

In this section, you will create a task, inspect the response metadata, and understand how Manus represents user prompts, outputs, and task state.

We'll test this with a simple query of 2 + 2 and get Manus to give us the answer.

In [47]:
# Create a simple task
headers = {
    "API_KEY": API_KEY,
    "Content-Type": "application/json"
}

data = {
    "prompt": "What is 2 + 2? Please provide a brief answer.",
    "agentProfile": "manus-1.5"
}

response = requests.post(
    f"{BASE_URL}/tasks",
    headers=headers,
    json=data
)
response.raise_for_status()

task = response.json()
print("✓ Task created successfully!\n")
print(task)

✓ Task created successfully!

{
    'task_id': 'd2ZumFyvuKrmuxRZRsFwwF',
    'task_title': 'What is 2 + 2',
    'task_url': 'https://manus.im/app/d2ZumFyvuKrmuxRZRsFwwF'
}

For this basic mathematical question, Manus will detect the simplicity and switch to Chat mode, providing an immediate response without unnecessary computational overhead.

When you create a task, Manus returns several key pieces of information that are essential for tracking and managing your request:

1. task_id: A unique identifier that you'll use to reference this specific task
2. task_url: A direct link to view the task execution in the Manus web interface
3. task_title: An automatically generated title that summarizes your request

Make sure that you store these values when you create your initial tasks so that you can monitor and manage your tasks programatically.

In [48]:
# Extract important values
task_id = task["task_id"]
task_url = task["task_url"]

print(
    f"Task ID: {task_id}\n"
    f"Task URL: {task_url}\n"
    f"\n👉 Click the link above to see your task running in real-time!"
)

Task ID: d2ZumFyvuKrmuxRZRsFwwF
Task URL: https://manus.im/app/d2ZumFyvuKrmuxRZRsFwwF

👉 Click the link above to see your task running in real-time!

Now that we've created our first task, let's learn a bit more about the different agent profiles that we can use.

### Manus-1.5 and Manus-1.5-lite

We recently introduced Manus 1.5, representing a significant leap forward in AI agent capabilities. This re-architected engine delivers nearly four times faster performance—tasks that once took 15 minutes now complete in under 4 minutes—while achieving 15% better task quality and 6% higher user satisfaction.

You can toggle between the two different versions of `Manus-1.5` in this notebook

1. `manus-1.5-lite` : This is our full-featured agent with maximum capability. We recommend this for more tasks that require more complex reasoning and agentic capabilities
2. `manus-1.5` : This is a streamlined version that's optimized for cost efficiency 

Our new agent modes also come with new updates such as

- **Expanded Context Management**: We've added the ability for Manus to maintain coherence across longer conversations with what we're calling unlimited context
- **Web Development Capabilities** We've built in the ability for Manus to create whole entire websites and host them with a full feature set of tools such as a database and backend server.

This means that you can now tackle larger and more complex problems without losing important details.

After creating a task, the next natural question is: How do you know when it’s finished? 

The following section covers how to repeatedly poll the Manus API to track a task’s progress.

## Polling for Task Status

Tasks in Manus run asynchronously, which means they may take time to complete, transition through multiple states, or pause for user input. 

Polling allows you to programmatically check task status and respond based on whether it is running, completed, pending, or in error.



One easy way to poll for our task completion is to use this endpoint and check for the `status` and see if it's transitioned from `running` to another state.

We have three possible states 

1. `error` : Manus has encountered an unrecoverable error. In this case, we have the option for you to get a full refund of the credits
2. `completed` : This indicates that Manus has finished its task 
3. `pending` : This indicates that Manus is pending your user response and needs your input

Let's see an example of this in action where we explicitly force Manus to get into a `pending` state

In [49]:
# Create a simple task
headers = {
    "API_KEY": API_KEY,
    "Content-Type": "application/json"
}

data = {
    "prompt": "Look up the weather in New York City next week and then ask me a question about it. Make sure that you wait for my response before you continue",
    "agentProfile": "manus-1.5-lite",
    "taskMode":"agent"
}

response = requests.post(
    f"{BASE_URL}/tasks",
    headers=headers,
    json=data
)
response.raise_for_status()

task_response = response.json()
task_id = task_response["task_id"]
print("✓ Task created successfully!\n")
print(task_response)

✓ Task created successfully!

{
    'task_id': 'irn7c6DYw5pNgiVLa8xiWC',
    'task_title': 'Weather in New York City Next Week?',
    'task_url': 'https://manus.im/app/irn7c6DYw5pNgiVLa8xiWC'
}

In [50]:
import time

POLLING_INTERVAL = 20

while True:
    url = f"https://api.manus.ai/v1/tasks/{task_id}"
    headers = {"API_KEY": API_KEY}
    response = requests.get(url, headers=headers)
    task = response.json()
    task_url = task["metadata"]["task_url"]
    task_id = task["id"]

    if task["status"] != "running":
        print(f"Task {task_id} is now {task['status']}, printing out the full task details")
        break

    print(f"Task is still running, waiting for {POLLING_INTERVAL} seconds before polling again. Check the current status at {task_url}")
    time.sleep(POLLING_INTERVAL)

print(task)

Task is still running, waiting for 20 seconds before polling again. Check the current status at 
https://manus.im/app/irn7c6DYw5pNgiVLa8xiWC

Task is still running, waiting for 20 seconds before polling again. Check the current status at 
https://manus.im/app/irn7c6DYw5pNgiVLa8xiWC

Task irn7c6DYw5pNgiVLa8xiWC is now pending, printing out the full task details

{
    'id': 'irn7c6DYw5pNgiVLa8xiWC',
    'object': 'task',
    'created_at': '1763341275',
    'updated_at': '1763341318',
    'status': 'pending',
    'model': 'manus-1.5-lite-agent',
    'metadata': {'task_url': 'https://manus.im/app/irn7c6DYw5pNgiVLa8xiWC'},
    'output': [
        {
            'id': 'qGrQJ3T0ytekuNZm3caywR',
            'status': 'completed',
            'role': 'user',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Look up the weather in New York City next week and then ask me a question about it. 
Make sure that you wait for my response before you continue'
                }
            ]
        },
        {
            'id': 'Ebbh4re2ZTAlkjAD48nepv',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Understood. I will first look up the weather forecast for New York City next week and 
then ask you a question about it, waiting for your response before proceeding.'
                }
            ]
        },
        {
            'id': 'bijp4xoZHKucd0zkBui1a1',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'I have successfully looked up the weather forecast for New York City for the upcoming 
week.\n\nHere is a summary of the forecast from Monday, November 17th, to Sunday, November 23rd:\n\n| Day | 
High/Low (°F) | Condition |\n| :--- | :--- | :--- |\n| **Mon 17** | 44° / 36° | Partly Cloudy / Clear skies |\n| 
**Tue 18** | 46° / 40° | Partly cloudy (AM) / Cloudy (PM) |\n| **Wed 19** | 47° / 41° | AM Clouds/PM Sun |\n| **Thu
20** | 47° / 46° | Cloudy / Overcast |\n| **Fri 21** | 54° / 50° | Cloudy with showers |\n| **Sat 22** | 54° / 43° 
| Partly cloudy |\n| **Sun 23** | 52° / 45° | Partly cloudy / Mostly cloudy with a few showers |\n\nBased on this 
information, I have a question for you:\n\n**The forecast indicates that Friday, November 21st, will be the warmest
day of the week with a high of 54°F and a 61% chance of showers. If you were planning an outdoor activity for that 
day, would you proceed with your plans, or would the high chance of rain cause you to reschedule?**'
                }
            ]
        }
    ],
    'credit_usage': 11
}

We can also push to existing sessions

In [51]:
# Create a simple task
headers = {
    "API_KEY": API_KEY,
    "Content-Type": "application/json"
}

data = {
    "prompt": "I'd probably look to cancel. What should i pack for a trip there over the next week?",
    "agentProfile": "manus-1.5-lite",
    "taskMode":"agent",
    "taskId": task_id
}

response = requests.post(
    f"{BASE_URL}/tasks",
    headers=headers,
    json=data
)
response.raise_for_status()

task_response = response.json()
task_id = task_response["task_id"]
print("✓ Task created successfully!\n")
print(task_response)

✓ Task created successfully!

{
    'task_id': 'irn7c6DYw5pNgiVLa8xiWC',
    'task_title': 'Weather in New York City Next Week',
    'task_url': 'https://manus.im/app/irn7c6DYw5pNgiVLa8xiWC'
}

In [52]:
import time

POLLING_INTERVAL = 20

while True:
    url = f"https://api.manus.ai/v1/tasks/{task_id}"
    headers = {"API_KEY": API_KEY}
    response = requests.get(url, headers=headers)
    task = response.json()
    task_url = task["metadata"]["task_url"]
    task_id = task["id"]

    if task["status"] != "running":
        print(f"Task {task_id} is now {task['status']}, printing out the full task details")
        break

    print(f"Task is still running, waiting for {POLLING_INTERVAL} seconds before polling again. Check the current status at {task_url}")
    time.sleep(POLLING_INTERVAL)

print(task)

Task irn7c6DYw5pNgiVLa8xiWC is now completed, printing out the full task details

{
    'id': 'irn7c6DYw5pNgiVLa8xiWC',
    'object': 'task',
    'created_at': '1763341275',
    'updated_at': '1763341423',
    'status': 'completed',
    'model': 'manus-1.5-lite-agent',
    'metadata': {'task_url': 'https://manus.im/app/irn7c6DYw5pNgiVLa8xiWC'},
    'output': [
        {
            'id': 'qGrQJ3T0ytekuNZm3caywR',
            'status': 'completed',
            'role': 'user',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Look up the weather in New York City next week and then ask me a question about it. 
Make sure that you wait for my response before you continue'
                }
            ]
        },
        {
            'id': 'Ebbh4re2ZTAlkjAD48nepv',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Understood. I will first look up the weather forecast for New York City next week and 
then ask you a question about it, waiting for your response before proceeding.'
                }
            ]
        },
        {
            'id': 'bijp4xoZHKucd0zkBui1a1',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'I have successfully looked up the weather forecast for New York City for the upcoming 
week.\n\nHere is a summary of the forecast from Monday, November 17th, to Sunday, November 23rd:\n\n| Day | 
High/Low (°F) | Condition |\n| :--- | :--- | :--- |\n| **Mon 17** | 44° / 36° | Partly Cloudy / Clear skies |\n| 
**Tue 18** | 46° / 40° | Partly cloudy (AM) / Cloudy (PM) |\n| **Wed 19** | 47° / 41° | AM Clouds/PM Sun |\n| **Thu
20** | 47° / 46° | Cloudy / Overcast |\n| **Fri 21** | 54° / 50° | Cloudy with showers |\n| **Sat 22** | 54° / 43° 
| Partly cloudy |\n| **Sun 23** | 52° / 45° | Partly cloudy / Mostly cloudy with a few showers |\n\nBased on this 
information, I have a question for you:\n\n**The forecast indicates that Friday, November 21st, will be the warmest
day of the week with a high of 54°F and a 61% chance of showers. If you were planning an outdoor activity for that 
day, would you proceed with your plans, or would the high chance of rain cause you to reschedule?**'
                }
            ]
        },
        {
            'id': 'zJGQWedbm8UsqwCr6f5q3U',
            'status': 'completed',
            'role': 'user',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': "I'd probably look to cancel. What should i pack for a trip there over the next week?"
                }
            ]
        },
        {
            'id': 'c6K2zNSE4GBIX0rR3qkHMX',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': "That is a prudent decision regarding the outdoor activity.\n\nBased on the New York 
City weather forecast for the upcoming week (November 17th to November 23rd), which features cool temperatures, 
wind, and a chance of rain, a strategic packing list should focus on **layers, warmth, and water 
resistance**.\n\n### Recommended Packing List for New York City (Next Week)\n\nThe weather will be consistently 
cool, with daily high temperatures ranging from the mid-40s to the mid-50s Fahrenheit, and overnight lows dropping 
into the 30s and 40s. The forecast also includes a significant chance of rain on Friday, November 21st, and a 
moderate chance on Sunday, November 23rd.\n\n| Category | Essential Items | Rationale |\n| :--- | :--- | :--- |\n| 
**Outerwear** | **Warm, waterproof or water-resistant jacket** (e.g., a park

Now that we've seen polling and task creation in action, let's create a few reusable functions.

In [43]:
def create_task(prompt, agent_profile="manus-1.5-lite", task_mode="agent", **kwargs):
    """Create a Manus task and return task details."""
    headers = {
        "API_KEY": API_KEY,
        "Content-Type": "application/json"
    }
    
    data = {
        "prompt": prompt,
        "agentProfile": agent_profile,
        "taskMode": task_mode,
        **kwargs
    }
    
    response = requests.post(f"{BASE_URL}/tasks", headers=headers, json=data)
    response.raise_for_status()
    
    return response.json()

def poll_and_wait(task_id, api_key, polling_interval=20):
    """Poll a task until completion and return final state."""
    while True:
        url = f"https://api.manus.ai/v1/tasks/{task_id}"
        headers = {"API_KEY": api_key}
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        task = response.json()
        task_url = task["metadata"]["task_url"]
        status = task["status"]
        
        if status != "running":
            print(f"Task {task_id} is now {status}")
            return task
        
        print(f"Task is still running. Checking again in {polling_interval}s. Status: {task_url}")
        time.sleep(polling_interval)

In [45]:
# Poll for our simple math task
task_response = create_task("Can you look up the weather in New York City?")
completed_task = poll_and_wait(task_response["task_id"], API_KEY)
print("\nFinal task details:")
print(completed_task)

Task is still running. Checking again in 20s. Status: https://manus.im/app/GcEutMeTsnjePR7gaxs9Sd

Task GcEutMeTsnjePR7gaxs9Sd is now completed

Final task details:

{
    'id': 'GcEutMeTsnjePR7gaxs9Sd',
    'object': 'task',
    'created_at': '1763286857',
    'updated_at': '1763286870',
    'status': 'completed',
    'model': 'manus-1.5-lite-agent',
    'metadata': {'task_url': 'https://manus.im/app/GcEutMeTsnjePR7gaxs9Sd'},
    'output': [
        {
            'id': 'rNS9XY5iyiOjaNTNMN59tR',
            'status': 'completed',
            'role': 'user',
            'type': 'message',
            'content': [{'type': 'output_text', 'text': 'Can you look up the weather in New York City?'}]
        },
        {
            'id': 'p2JYxrdmUduTuyhrCQZVdo',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Understood, I will look up the weather in New York City for you.'
                }
            ]
        },
        {
            'id': 'yI8AKaO2qzuektGMl1BbG6',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'The current weather in New York City is **48°F (8°C)**, with a **"Feels Like" 
temperature of 43°F (6°C)**.\n\nThe conditions are **Cloudy**, with a chance of rain expected later in the day or 
overnight.\n\nHere are some additional details:\n*   **Humidity:** Approximately 53%\n*   **Wind:** West-Northwest 
at 10 to 11 mph'
                }
            ]
        }
    ],
    'credit_usage': 8
}

## Using Connectors in Tasks

Manus ships with connectors, here's how you use it well

In [41]:
# Define headers
headers = {
    "API_KEY": API_KEY,
    "Content-Type": "application/json"
}

# Define request data
data = {
    "prompt": "What was the most recently modified page in Notion?",
    "taskMode": "agent",
    "agentProfile": "manus-1.5-lite",
    "connectors": ["9c27c684-2f4f-4d33-8fcf-51664ea15c00"]
}

# Make the request
response = requests.post(
    f"{BASE_URL}/tasks",
    headers=headers,
    json=data
)

response.raise_for_status()

# Print result
task = response.json()
task_id = task["task_id"]
task_url = task["task_url"]
print("✓ Task created successfully!\n")
print(task)

✓ Task created successfully!

{
    'task_id': 'LaVZGFByWvE7f6UN5iHK5P',
    'task_title': 'Most recently modified page in Notion?',
    'task_url': 'https://manus.im/app/LaVZGFByWvE7f6UN5iHK5P'
}

In [42]:
def poll_and_wait(task_id, api_key, polling_interval=20):
    """Poll a task until it's no longer running and return the final task state."""
    while True:
        url = f"https://api.manus.ai/v1/tasks/{task_id}"
        headers = {"API_KEY": api_key}
        response = requests.get(url, headers=headers)
        task = response.json()
        task_url = task["metadata"]["task_url"]
        
        if task["status"] != "running":
            print(f"Task {task_id} is now {task['status']}, printing out the full task details")
            return task
        
        print(f"Task is still running, waiting for {polling_interval} seconds before polling again. Check the current status at {task_url}")
        time.sleep(polling_interval)

# Usage
task = poll_and_wait(task_id, API_KEY)
print(task)

Task is still running, waiting for 20 seconds before polling again. Check the current status at 
https://manus.im/app/LaVZGFByWvE7f6UN5iHK5P

Task is still running, waiting for 20 seconds before polling again. Check the current status at 
https://manus.im/app/LaVZGFByWvE7f6UN5iHK5P

Task is still running, waiting for 20 seconds before polling again. Check the current status at 
https://manus.im/app/LaVZGFByWvE7f6UN5iHK5P

Task LaVZGFByWvE7f6UN5iHK5P is now completed, printing out the full task details

{
    'id': 'LaVZGFByWvE7f6UN5iHK5P',
    'object': 'task',
    'created_at': '1763283500',
    'updated_at': '1763283557',
    'status': 'completed',
    'model': 'manus-1.5-lite-agent',
    'metadata': {'task_url': 'https://manus.im/app/LaVZGFByWvE7f6UN5iHK5P'},
    'output': [
        {
            'id': 'GSWCUOiA4fbUjVdMQgnTKc',
            'status': 'completed',
            'role': 'user',
            'type': 'message',
            'content': [{'type': 'output_text', 'text': 'What was the most recently modified page in Notion?'}]
        },
        {
            'id': 'cZP9AVuwuoYhKmvdRURCfv',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'The most recently modified page in your Notion workspace, based on the search results,
is:\n\n*   **Title:** [ST 1.6] Schedule Summary Page\n*   **URL:** 
https://www.notion.so/2a27ff657cfa804a92dafe93cf50bc65\n*   **Last Modified:** 2025-11-14'
                }
            ]
        }
    ],
    'credit_usage': 14
}

## Next Steps

In this notebook, we've taken our first steps into the Manus API. 

We've learned how to create a task, poll for its status, and retrieve the results. 

We've also seen how to use connectors to extend Manus's capabilities. This foundational knowledge is the building block for creating more complex and powerful workflows.

In the next notebook, we'll explore how to upload files and provide them as context to your Manus tasks. 